In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR

In [ ]:
train = pd.read_csv("train.csv",parse_dates=["Date"])
test = pd.read_csv("test.csv",parse_dates=["Date"])
store = pd.read_csv("store.csv")
sample = pd.read_csv("sample_submission.csv")
sample = sample.drop(["Sales"],axis=1)

print("If no data, store is open as most of the days it's open")
train =train.fillna(1)
test=test.fillna(1)

print("Consider only open stores for training. Closed stores wont count into the score")
train = train[train["Open"] != 0]
print("Use only Sales bigger then zero")
train = train[train["Sales"] > 0]


In [ ]:
train = pd.merge(train,store,on="Store")
test = pd.merge(test,store,on="Store")

In [ ]:
print(" Nulls in train data: \n")
print(train.isna().sum())
print("\n Nulls in test data: \n")
print(test.isna().sum())

In [ ]:
def feature_engg(data):
    mappings = {'0':"0", 'a':"1", 'b':"2", 'c':"3", 'd':"4"}
    mappings1 = {'0':"0", 'Jan,Apr,Jul,Oct':"1", 'Feb,May,Aug,Nov':"2", 'Mar,Jun,Sept,Dec':"3"}
    data["StoreType"].replace(mappings, inplace=True)
    data["PromoInterval"].replace(mappings1, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['WeekOfYear'] = data.Date.dt.weekofyear
    data['Day_Week'] = data.Date.dt.weekday_name
    mappings2 = {'Sunday':"0", 'Monday':"1", 'Tuesday':"2", 'Wednesday':"3", 'Thursday':"4", 'Friday':"5", 'Saturday':"6"}
    data["Day_Week"].replace(mappings2, inplace=True)
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + \
        (data.Month - data.CompetitionOpenSinceMonth)
    data['CompetitionOpen'] = data.CompetitionOpen.apply(lambda x: x if x > 0 else 0)
    data['PrvsHoliday'] = np.where((data["StateHoliday"].shift(1)==1) | (data["SchoolHoliday"].shift(1)==1),1,0)
    data['NxtHoliday'] = np.where((data["StateHoliday"].shift(-1)==1) | (data["SchoolHoliday"].shift(-1)==1),1,0)
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + \
        (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['PromoOpen'] = data["PromoOpen"].apply(lambda x: x if x > 0 else 0)
    data.loc[data.Promo2SinceYear == 0, 'PromoOpen'] = 0
    
    return data

In [ ]:
train = feature_engg(train)
test = feature_engg(test)

In [ ]:
#train.isna().sum()

In [ ]:
plt.hist(train["CompetitionDistance"].dropna())

In [ ]:
train["CompetitionDistance"].fillna(np.median(train["CompetitionDistance"].dropna()),inplace=True)

In [ ]:
plt.hist(train["CompetitionOpenSinceMonth"].dropna())

In [ ]:
train =train.fillna(0)
test =test.fillna(0)

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

In [ ]:
#def type_conv(df):
#    df['Store'] = df['Store'].astype('category')
#    df['DayOfWeek'] = df['Store'].astype('category')
#    df['StateHoliday'] = df['StateHoliday'].astype('category')
#    df['SchoolHoliday'] = df['SchoolHoliday'].astype('category')
#    df['Assortment'] = df['Assortment'].astype('category')
#    df['StoreType'] = df['StoreType'].astype('category')
#    df['Promo2']= df['Promo2'].astype('category')
#    df['PromoInterval']= df['Promo2'].astype('category')
#    return df

In [ ]:
#train = type_conv(train)
#test = type_conv(test)

In [ ]:
train = train.drop(["CompetitionOpenSinceMonth","CompetitionOpenSinceYear","Customers","Promo2SinceWeek","Promo2SinceYear"],axis=1)
test = test.drop(["CompetitionOpenSinceMonth","CompetitionOpenSinceYear","Promo2SinceWeek","Promo2SinceYear"],axis=1)

In [ ]:
train.shape[0]

In [ ]:
df = pd.get_dummies(train[["StoreType","Assortment",
                          "PromoInterval"]],prefix_sep="_",drop_first=True)
df1 = pd.get_dummies(test[["StoreType","Assortment",
                          "PromoInterval"]],prefix_sep="_",drop_first=True)

In [ ]:
train = pd.concat([train,df],axis=1)
test = pd.concat([test,df1],axis=1)

In [ ]:
def rmspe(y, yhat):
    return np.sqrt(np.mean(((y - yhat)/y) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

In [ ]:
X_train = train.drop(["Sales","Date","Open",],axis=1)
y_train = train["Sales"]
X_test = test.drop(["Id","Date","Open",],axis=1)

In [ ]:
scaler = StandardScaler()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=15)
X_train_scale = scaler.fit_transform(X_train)
X_val_scale = scaler.transform(X_val)
X_test_scale = scaler.transform(X_test)

In [ ]:
inner_cv = KFold(n_splits=4, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=4, shuffle=True, random_state=0)

In [ ]:
import xgboost as xgb
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.1,
          "max_depth": 10,
          "subsample": 0.85,
          "colsample_bytree": 0.4,
          "min_child_weight": 6,
          "silent": 1,
          "thread": 1,
          "seed": 1301
          }
num_boost_round = 2000


y_train = np.log(y_train)
y_val1 = np.log(y_val)
dtrain = xgb.DMatrix(X_train_scale, y_train)
dvalid = xgb.DMatrix(X_val_scale, y_val1)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

In [ ]:
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=10, \
  feval=rmspe_xg, verbose_eval=True)


In [ ]:
print("Validating")
yhat1 = gbm.predict(xgb.DMatrix(X_val_scale))
error = rmspe(y_val, np.exp(yhat1))
print('RMSPE: {:.6f}'.format(error))

print("Make predictions on the test set")
dtest = xgb.DMatrix(X_test_scale)
test_probs = gbm.predict(dtest)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators=50)
rfr.fit(X_train,y_train)

In [ ]:
print("Validating")
yhat = rfr.predict(X_val)
error = rmspe(y_val, np.exp(yhat))
print('RMSPE: {:.6f}'.format(error))

print("Make predictions on the test set")
test_probs_rfr = rfr.predict(X_test)

In [ ]:
rmspe(y_val, np.exp(yhat2))

In [ ]:
lsvr = LinearSVR(C=.01).fit(X_train_scale,y_train)
model = SelectFromModel(lsvr, prefit=True)
X_train_scale_sm = model.transform(X_train_scale)
print(X_train_scale_sm.shape)
X_test_scale_sm = model.transform(X_test_scale)
print(X_train_scale.shape)

In [ ]:
from sklearn.svm import SVR, LinearSVR
svr = LinearSVR(C=0.01)
svr.fit(X_train_scale_sm,y_train)

In [ ]:
X_val_scale_sm = model.transform(X_val_scale)
print("Validating")
yhat = svr.predict(X_val_scale_sm)
error = rmspe(y_val, np.exp(yhat))
print('RMSPE: {:.6f}'.format(error))

print("Make predictions on the test set")
test_probs_svr = svr.predict(X_test_scale_sm)

In [ ]:
regr = LinearRegression()
X_val_scale_sm = model.transform(X_val_scale)

In [ ]:
regr.fit(X_train_scale_sm,y_train)
print("Validating Linear Regression")
yhat_lr = regr.predict(X_val_scale_sm)
error = rmspe(y_val, np.exp(yhat_lr))
print('RMSPE: {:.6f}'.format(error))

print("Make predictions on the test set")
test_probs_lr = regr.predict(X_test_scale_sm)

In [ ]:
from sklearn.linear_model import Lasso,Ridge
from sklearn.metrics.scorer import make_scorer
my_scorer = make_scorer(rmspe, greater_is_better=False)

In [ ]:
p_grid = {'alpha': [1e-3,1e-2,1e-1]}
lasso = Lasso()
# Non_nested parameter search and scoring
gs = GridSearchCV(estimator=lasso, param_grid=p_grid,
 cv=inner_cv, scoring=my_scorer, return_train_score=True)
gs.fit(X_train_scale_sm, y_train)

In [ ]:
print("Validating Lasso")
yhat_lr = gs.predict(X_val_scale_sm)
error = rmspe(y_val, np.exp(yhat_lr))
print('RMSPE: {:.6f}'.format(error))
print('Best params are: {0}'.format(gs.best_params_))

print("Make predictions on the test set")
test_probs_lr = regr.predict(X_test_scale_sm)

In [ ]:
p_grid = {'alpha': [1e-3,1e-2,1e-1]}
ridge = Ridge()
# Non_nested parameter search and scoring
gs = GridSearchCV(estimator=ridge, param_grid=p_grid,
 cv=inner_cv, scoring=my_scorer,return_train_score=True)
gs.fit(X_train_scale_sm, y_train)

In [ ]:
print("Validating Ridge")
yhat_lr = gs.predict(X_val_scale_sm)
error = rmspe(y_val, np.exp(yhat_lr))
print('RMSPE: {:.6f}'.format(error))
print('Best params are: {0}'.format(gs.best_params_))
print("Make predictions on the test set")
test_probs_lr = regr.predict(X_test_scale_sm)

In [ ]:
# Make Submission
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.exp(test_probs), "open":test['Open']})
#result["Sales"]=np.where(result["open"]==0,0,result["Sales"])
result=result.drop("open",axis=1)
result.to_csv("submission1.csv", index=False)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from livelossplot import PlotLossesKeras
import tensorflow as tf
from keras import optimizers
from keras import regularizers
import keras.backend as K
from keras.layers import LSTM,GRU
from keras.layers.core import Dense, Activation, Dropout

In [ ]:
def RMSPE_for_Keras(y, yhat):
    return K.sqrt(K.mean(((y - yhat)/y) ** 2))

In [ ]:
s = tf.InteractiveSession()
model = Sequential()
model.add(Dense(50, input_dim=np.shape(X_train_scale)[1],activation='relu',kernel_initializer = 'uniform'))
model.add(Dropout(0.2))
model.add(Dense(25,activation='relu',kernel_initializer = 'uniform'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='relu',kernel_initializer = 'uniform'))
model.add(Dense(1))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='mean_absolute_percentage_error', optimizer='adam')
model.fit(X_train_scale, y_train, epochs=20,batch_size=200, verbose=1,callbacks=[PlotLossesKeras()],validation_data=(
    X_val_scale, y_val1))

In [ ]:
print("Validating")
yhat_mlp = model.predict(X_val_scale)
yhat_mlp = [x[0] for x in yhat_mlp]
error = rmspe(y_val, np.exp(yhat_mlp))
print('RMSPE: {:.6f}'.format(error))

print("Make predictions on the test set")
test_probs_mlp = model.predict(X_test_scale)

In [ ]:
X_train_scale_rs = np.reshape(X_train_scale, (X_train_scale.shape[0],X_train_scale.shape[1],1))
X_val_scale_rs = np.reshape(X_val_scale, (X_val_scale.shape[0],X_val_scale.shape[1],1))
X_test_scale_rs = np.reshape(X_test_scale, (X_test_scale.shape[0],X_test_scale.shape[1],1))